# Getting PBMC data from:


Donor1 - SRR8718858
Donor1 - SRR8718859
Donor2 - SRR8718860
Donor2 - SRR8718861
Donor3 - SRR8718862
Donor3 - SRR8718863


In [ ]:
%%bash 

cd /input_dir/crispr_analysis/input_reads

prefetch -o /input_dir/crispr_analysis/input_reads/pbmc/SRR8718858.sra SRR8718858
prefetch -o /input_dir/crispr_analysis/input_reads/pbmc/SRR8718860.sra SRR8718860


In [ ]:
%%bash

parallel-fastq-dump -t 8 --tmpdir /input_dir/crispr_analysis/temp/ \
    -s /input_dir/crispr_analysis/input_reads/pbmc/SRR8718858.sra \
    --dumpbase --clip --readids \
    --read-filter pass --split-files --origfmt \
    --outdir /input_dir/crispr_analysis/input_reads/pbmc

parallel-fastq-dump -t 8 --tmpdir /input_dir/crispr_analysis/temp/ \
    -s /input_dir/crispr_analysis/input_reads/pbmc/SRR8718860.sra \
    --dumpbase --clip --readids \
    --read-filter pass --split-files --origfmt \
    --outdir /input_dir/crispr_analysis/input_reads/pbmc


In [ ]:
%%bash
mkdir /input_dir/crispr_analysis/alignment_out/pbmc/


In [ ]:
%%bash
cd /input_dir/crispr_analysis/input_reads/pbmc
#Attempt doing a sample from landscape
salmon alevin -l ISR \
              -1 SRR8718858_pass_1.fastq \
              -2 SRR8718858_pass_2.fastq \
              --chromium \
              -i /input_dir/corona_analysis/annotations/human/salmon_ann/salmon_hg38_index \
              -p 3 \
              --mrna /input_dir/corona_analysis/annotations/human/gencode_mt.txt \
              --rrna /input_dir/corona_analysis/annotations/human/rRNA_ensembl.txt \
              -o /input_dir/crispr_analysis/alignment_out/pbmc/donor_1_1 \
              --tgMap /input_dir/corona_analysis/annotations/human/salmon_grch38_gencode_tran2gene.txt


In [ ]:

salmon alevin -l ISR \
              -1 SRR8718860_pass_1.fastq \
              -2 SRR8718860_pass_2.fastq \
              --chromium \
              -i /input_dir/corona_analysis/annotations/human/salmon_ann/salmon_hg38_index \
              -p 10 \
              --mrna /input_dir/corona_analysis/annotations/human/gencode_mt.txt \
              --rrna /input_dir/corona_analysis/annotations/human/rRNA_ensembl.txt \
              -o /input_dir/crispr_analysis/alignment_out/pbmc/donor_2_1 \
              --tgMap /input_dir/corona_analysis/annotations/human/salmon_grch38_gencode_tran2gene.txt


In [ ]:
%%bash
#Get whitelist for version 10x v2
curl -s -L https://github.com/BUStools/getting_started/releases/download/velocity_tutorial/10xv2_whitelist.txt > \
    /input_dir/corona_analysis/annotations/human/scRNA_10x_v2_whitelist.txt
    

In [ ]:
%%bash

#Align reads against human transcriptome for PBMC from donors  
cd /input_dir/crispr_analysis/alignment_out/pbmc/
#Ctlr1
STAR --runMode alignReads      \
     --genomeDir /input_dir/corona_analysis/annotations/human/STAR_ix \
     --outSAMtype BAM SortedByCoordinate --runThreadN 14 \
     --outFileNamePrefix /input_dir/crispr_analysis/alignment_out/pbmc/donor_1_1_star \
     --outTmpDir /input_dir/crispr_analysis/temp/star1 \
     --outReadsUnmapped Fastx --limitBAMsortRAM 90000000000 \
     --outWigType wiggle --quantMode TranscriptomeSAM  \
     --soloType CB_UMI_Simple --soloCBwhitelist /input_dir/corona_analysis/annotations/human/scRNA_10x_v2_whitelist.txt \
     --soloFeatures Gene --soloOutFileNames pbmc_scRNA_d1_solo_ \
    --readFilesIn SRR8718860_pass_1.fastq \
                   SRR8718858_pass_2.fastq



In [ ]:
%%bash

cd /input_dir/corona_analysis/alignment_out/kidney_scRNA_control/
#Quantify expression in control BALF samples 1-3

#Ctlr1
rsem-calculate-expression --paired-end \
                       --alignments \
                       -p 14 --no-bam-output \
                       kidney_scRNA_control1_Aligned.toTranscriptome.out.bam \
                      /input_dir/corona_analysis/annotations/human/RSEM_ix \
                       kidney_scRNA_rsem_control1_